In [1]:
import pandas as pd
import numpy as np

#carga datasets
df_apre = pd.read_csv("../data/interim/aprehendidos_detenidos/aprehendidos_limpio_final.csv")
df_911 = pd.read_csv("../data/processed/ecu911/ecu911_parroquia_dia_conteo.csv")

C:\Users\fernando\AppData\Local\Temp\ipykernel_25260\2617150519.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_apre = pd.read_csv("../data/interim/aprehendidos_detenidos/aprehendidos_limpio_final.csv")


In [2]:
# Normalizacion de fechas
df_911["fecha"] = pd.to_datetime(df_911["fecha_dia"], errors="coerce").dt.date
df_apre["fecha"] = pd.to_datetime(df_apre["fecha"], errors="coerce").dt.date

In [3]:
#union de datasets por celdas 
df_union = df_apre.merge(
    df_911,
    on=["lat_grid", "lon_grid", "fecha"],
    how="outer",  # importante para no perder celdas sin datos
    suffixes=("_apre", "_911")
)

In [4]:
if "mes" not in df_union.columns:
    df_union["mes"] = pd.to_datetime(df_union["fecha"]).dt.month

if "dia" not in df_union.columns:
    df_union["dia"] = pd.to_datetime(df_union["fecha"]).dt.day

if "dia_semana" not in df_union.columns:
    df_union["dia_semana"] = pd.to_datetime(df_union["fecha"]).dt.dayofweek


In [5]:
# ELIMINAR FRANJA HORARIA PORQUE NO TIENE DATOS ÚTILES
if "franja_horaria" in df_union.columns:
    df_union.drop(columns=["franja_horaria"], inplace=True)

In [6]:
features = [
    "lat_grid", "lon_grid",
    "mes", "dia", "dia_semana",
    "franja_horaria",
    "conteo_delitos",
    "conteo_delitos_graves",
    "conteo_llamadas_riesgo"
]

In [7]:
# asegurarse de que las columnas existan
features = [c for c in features if c in df_union.columns]

df_final = df_union[features].copy()

In [8]:
df_final["conteo_delitos"] = (
  pd.to_numeric(df_final["conteo_delitos"], errors="coerce")
  .fillna(0)
  .astype(int)
)

df_final["conteo_delitos_graves"] = (
  pd.to_numeric(df_final["conteo_delitos_graves"], errors="coerce")
  .fillna(0)
  .astype(int)
)

df_final["conteo_llamadas_riesgo"] = (
  pd.to_numeric(df_final["conteo_llamadas_riesgo"], errors="coerce")
  .fillna(0)
  .astype(int)
)

In [9]:
# guardar dataset final
df_final.to_csv("../data/processed/dataset_entrenamiento_final_final.csv", index=False)

print("Registros totales:", len(df_final))
print("Columnas:", df_final.columns.tolist())

Registros totales: 199646
Columnas: ['lat_grid', 'lon_grid', 'mes', 'dia', 'dia_semana', 'conteo_delitos', 'conteo_delitos_graves', 'conteo_llamadas_riesgo']
